# Estimating Social Interaction Reductions Induced By Social Distancing and School and Non-essential Business Closures

### Quantifying Social Interactions

**Zhaoyang R, Sliwinski MJ, Martire LM, Smyth JM.** Age differences in adults' daily social interactions: An ecological momentary assessment study. Psychol Aging. 2018;33(4):607–618. doi:10.1037/pag0000242

**Synopsis**: At the individual level, participants reported, on average, 2.4 social interactions at each momentary assessment (that is, approximately 12 social interactions per day). In terms of partner type, participants reported that 45% of their most recent social interactions were with one or more family members, 25% were with friends and 30% were with peripheral relationship partners. 

| Parameters | Values |
|---|---|
| Average Interactions a day | 12 |
| Adult Interaction Ratios | Family (40%), Friends (30), Other (30%) |
| Child Interaction Ratios | Family (45%), Friends (20), Other (35%) |


| Parameters | Ratio of Other Interactions |
|---|---|
| Kids | 70% school , 12% nonessential , 18% essential |
| Adults | 30% essential, 70% nonessential |



Article found at: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6113687/

Penn Medicine Predictive Healthcare Blog Post (relevant acronyms): http://predictivehealthcare.pennmedicine.org/2020/03/18/compare-chime.html

### Philadelphia, Pennsylvania Area Population Breakdown (WHAT IS THE SOURCE?)

**Everyone**: 3,605,688

**Adults, 18+**: 2,831,376

20% of population are children (<18)

### Parameters Based On Zhaoyang et al. 2018


In [ ]:
kid_to_adult_ratio = 0.2
num_adults = 100
num_kids = num_adults*kid_to_adult_ratio

# Average number of interaction by people a day
number_of_interactions = 12
num_kids_interactions = num_kids*number_of_interactions
num_adult_interactions = num_adults*number_of_interactions

# Adult ratios of interaction based on paper
mid_adults_family = 0.4
mid_adults_friends = 0.3
mid_adults_other = 1 - (mid_adults_family + mid_adults_friends)
adults_nonessential = mid_adults_other*0.3
adults_essentail = mid_adults_other-adults_nonessential

# Kids ratios of interaction based on paper
kids_family = 0.45
kids_friends = 0.2
kids_other = 1 - (kids_family + kids_friends)
school_ratio = 0.70 # 70% of a kids 'other' interaction besides family and friends.
school = kids_other*school_ratio
kids_nonessential = (kids_other - school)*0.4 #shouldn't the 0.4 be 0.12?
kids_essentail = (kids_other - school)-kids_nonessential

# Effect Methods Classes

In [5]:
from abc import ABC 


class AdministrativeRegion(ABC):
    
    def attributes(self):
        pass
    
    
class City(AdministrativeRegion):
    
    def attributes(self, 
                   name, 
                   sd_policy_dictionary, 
                   sd_policy,
                   num_adults, 
                   kid_to_adult_ratio, 
                   number_of_interactions, 
                   mid_adults_family, 
                   mid_adults_friends, 
                   adults_nonessential_rate, 
                   kids_family, 
                   kids_friends, 
                   school_ratio, 
                   kids_nonessential_rate):
        
        self.name = name
        self.sd_policy_dictionary = sd_policy_dictionary
        self.sd_policy = sd_policy
        
        self.num_adults = num_adults
        self.kid_to_adult_ratio = kid_to_adult_ratio
        self.number_of_interactions = number_of_interactions
        self.mid_adults_family = mid_adults_family
        self.mid_adults_friends = mid_adults_friends
        self.adults_nonessential_rate = adults_nonessential_rate
        self.kids_family = kids_family
        self.kids_friends = kids_friends
        self.school_ratio = school_ratio
        self.kids_nonessential_rate = kids_nonessential_rate
        
        self.sd_effectiviness = self.sd_policy_dictionary[self.sd_policy]
        self.num_adult_interactions = self.num_adults * self.number_of_interactions
        self.mid_adults_other = 1 - (self.mid_adults_family + self.mid_adults_friends)
        self.adults_nonessential_other = self.mid_adults_other * self.adults_nonessential_rate
        self.adults_essential_other = self.mid_adults_other - self.adults_nonessential_other
        
        self.num_kids = self.num_adults * self.kid_to_adult_ratio
        self.num_kids_interactions = self.num_kids * self.number_of_interactions
        self.kids_other = 1 - (self.kids_family + self.kids_friends)
        self.school = self.kids_other * self.school_ratio    
        self.kids_nonessential_other = (self.kids_other - self.school) * 0.4 #Shouldn't this be 0.12 per table above? #ToDo create new attribute for true value
        self.kids_essential_other = (self.kids_other - self.school) - self.kids_nonessential_other
        self.total_interactions = self.num_kids_interactions + self.num_adult_interactions

        return self
    

class SocialInteraction(ABC): 
  
    def effect(self): 
        pass
    

class SchoolClosures(SocialInteraction):
    
    def effect(self,
               AdministrativeRegion):
        
        self.num_kids_interactions = AdministrativeRegion.num_kids_interactions
        self.kids_other = AdministrativeRegion.kids_other
        self.school_ratio = AdministrativeRegion.school_ratio
        
        return self.num_kids_interactions * self.kids_other * self.school_ratio
    

class NonessentialBusinessClosures(SocialInteraction):
    
    def effect(self,
               AdministrativeRegion):
        
        self.num_adult_interactions = AdministrativeRegion.num_adult_interactions
        self.num_kids_interactions = AdministrativeRegion.num_kids_interactions
        self.adults_nonessential_other= AdministrativeRegion.adults_nonessential_other
        self.kids_nonessential_other = AdministrativeRegion.kids_nonessential_other
        
        self.kid_reduc = self.num_kids_interactions * self.kids_nonessential_other
        self.adult_reduc = self.num_adult_interactions * self.adults_nonessential_other
        
        return self.kid_reduc + self.adult_reduc
    

class SocialDistancing(SocialInteraction):
    
    def effect(self,
               AdministrativeRegion):
        
        self.num_adult_interactions = AdministrativeRegion.num_adult_interactions
        self.num_kids_interactions = AdministrativeRegion.num_kids_interactions
        self.kids_nonessential = AdministrativeRegion.kids_nonessential_other
        self.adults_nonessential = AdministrativeRegion.adults_nonessential_other
        self.kids_friends= AdministrativeRegion.kids_friends
        self.mid_adults_friends= AdministrativeRegion.mid_adults_friends
        self.sd_effectiviness = AdministrativeRegion.sd_effectiviness
        
        self.kid_reduc = self.num_kids_interactions * (self.kids_nonessential + self.kids_friends) * self.sd_effectiviness
        self.adult_reduc = self.num_adult_interactions * (self.adults_nonessential + self.mid_adults_friends) * self.sd_effectiviness
        
        return self.kid_reduc + self.adult_reduc
    

class Simulation(ABC):
    
    def run(self): 
        pass
    
    
class SimulateSocialInteractions(Simulation):
    
    def run(self, AdministrativeRegion):
        
        self.AdministrativeRegion = AdministrativeRegion
        
        self.kids_pc_reduc = SchoolClosures().effect(self.AdministrativeRegion)
        self.nonessential_reduc = NonessentialBusinessClosures().effect(self.AdministrativeRegion)
        self.sd_reduc= SocialDistancing().effect(self.AdministrativeRegion)
        
        self.pc_reduc_rate = 1 - (self.AdministrativeRegion.total_interactions - self.kids_pc_reduc) / self.AdministrativeRegion.total_interactions
        self.nonessential_reduc_rate = 1 - (self.AdministrativeRegion.total_interactions - self.nonessential_reduc) / self.AdministrativeRegion.total_interactions
        self.sd_reduc_rate = 1 - (self.AdministrativeRegion.total_interactions - self.sd_reduc) / self.AdministrativeRegion.total_interactions
        
        self.pc_non_sd = self.pc_reduc_rate + self.nonessential_reduc_rate + self.sd_reduc_rate
        
        self.results = {"AdministrativeRegion":self.AdministrativeRegion.name,
                        "kids_pc_reduc":self.kids_pc_reduc, 
                        'nonessential_reduc':self.nonessential_reduc, 
                        "sd_reduc":self.sd_reduc, 
                        "pc_reduc_rate":self.pc_reduc_rate, 
                        "nonessential_reduc_rate":self.nonessential_reduc_rate, 
                        "sd_reduc_rate":self.sd_reduc_rate,
                        "pc_non_sd":self.pc_non_sd}
        
        return self.results
        

### Instantiation

In [6]:
philly = City().attributes(name="Philadelphia, Pennsylvania", 
                           sd_policy_dictionary={'six_feet_guidelines':0.30, 
                                                "stay_at_home": 0.75}, 
                           sd_policy="stay_at_home",
                           num_adults=2831376, 
                           kid_to_adult_ratio=0.2, 
                           number_of_interactions=12, 
                           mid_adults_family=0.4, 
                           mid_adults_friends=0.3, 
                           adults_nonessential_rate=0.3, 
                           kids_family=0.45, 
                           kids_friends=0.2, 
                           school_ratio=0.7, 
                           kids_nonessential_rate=0.4)

SimulateSocialInteractions().run(philly)

{'AdministrativeRegion': 'Philadelphia, Pennsylvania',
 'kids_pc_reduc': 1664849.0879999998,
 'nonessential_reduc': 3343288.7808000008,
 'sd_reduc': 11171477.1456,
 'pc_reduc_rate': 0.04083333333333328,
 'nonessential_reduc_rate': 0.08199999999999996,
 'sd_reduc_rate': 0.274,
 'pc_non_sd': 0.39683333333333326}